In [ ]:
# Example FCPG point query script
# Python 3, see imports for dependencies

import rasterio as rs
import json
import urllib.request
import pandas as pd
import numpy as np
import itertools

In [ ]:
data = pd.read_csv('FCPG_snap.csv')
df = pd.DataFrame(data, columns=['FCPG_X','FCPG_Y'])
print(df)

In [ ]:
jsonURL = "https://prod-is-usgs-sb-prod-publish.s3.amazonaws.com/5f3ab12082ce8df5b6c4076a/params.json"

In [ ]:
# load the database of FCPG parameters and URLS directly from ScienceBase.
params = json.load(urllib.request.urlopen(jsonURL))

In [ ]:
print(params['parameters']) # show all available parameters

#### Theo's method to query at one point

In [ ]:
#URL = params['urls']['tmin'] # get the URL for tmin FCPG

In [ ]:
# # define point to query, this should be in USGS Albers Equal Conic (EPSG 5070, https://epsg.io/5070-1252).
# x = -1106211.1
# y = 2318491.8
# band = 1 # query the first band of the raster

In [ ]:
#xy = (x,y)

In [ ]:
# with rs.open(URL) as src:
#     for i in src.sample([xy], band):
#         print(f'Mean minimum upstream temperature: {i[0]}')

#### New method to query all points in csv for one param at a time

In [ ]:
# x=np.array(df.iloc[:,0])
# y=np.array(df.iloc[:,1])
# band = 1 #query the first band of the raster

In [ ]:
# xy = list(zip(x,y))

In [ ]:
# with rs.open(URL) as src:
#     tmin=[]
#     for i in src.sample([xy][0], band):
#         x=i
#         tmin.append(x)

In [ ]:
# labels = ['tmin']

In [ ]:
# tmin=pd.DataFrame(data=tmin, columns=labels)

In [ ]:
# data_out = pd.concat([data, tmin], axis=1)

In [ ]:
#data_out

#### Query all points in csv for all params

In [ ]:
x=np.array(df.iloc[:,0])
y=np.array(df.iloc[:,1])
band = 1 #query the first band of the raster

In [ ]:
xy = list(zip(x,y))

In [ ]:
param_list = params['parameters']

In [ ]:
URL_all=[]
for i in param_list:
    urls = params['urls'][i]
    URL_all.append(urls)

In [ ]:
URL_all

### **OBTAIN DATA**

In [ ]:
data_out=[]

In [ ]:
for i in range(len(URL_all)):
    with rs.open(URL_all[i]) as src:
        for j in src.sample([xy][0], band):
            data_out.append(j)

In [ ]:
data_out_np = np.array(data_out)
data_out_rs = data_out_np.reshape(27,57454)    # replace the value after 27 with the number of data points
data_out_rs

In [ ]:
print(data_out_rs.shape)

In [ ]:
data_out_df = pd.DataFrame(data=data_out_rs.T, columns=param_list)
data_out_df

In [ ]:
data_reindexed = data.reset_index(drop=True)

In [ ]:
data_reindexed

In [ ]:
data_merged = pd.concat([data_reindexed, data_out_df], axis=1)
data_merged

In [ ]:
data_merged.to_csv('FCPG_snap_output_12_5_24.csv')